<a href="https://colab.research.google.com/github/aparnavinayankozhipuram/RAG-EVALUATION/blob/main/15_April_MIT_RAG_Score_Evaluation_2nd_query_9th_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk rouge-score
!pip install scikit-learn


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=71f853658dc6f07e4956ccf3be6b5a42f7e2a86adacf0f74c5f2edcdbe40b689
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [13]:
import nltk
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure necessary NLTK data is downloaded
# The original line was: nltk.download('punkt')
nltk.download('punkt_tab') # Download punkt_tab instead of just punkt


def evaluate_rag_model(predictions, references):
    """
    Evaluate RAG model using BLEU and ROUGE scores.

    Args:
    predictions (list of str): The generated text from the RAG model.
    references (list of str): The reference ground truth text.

    Returns:
    dict: BLEU and ROUGE scores.
    """
    # BLEU score
    bleu_scores = []
    for prediction, reference in zip(predictions, references):
        reference_tokens = nltk.word_tokenize(reference.lower())
        prediction_tokens = nltk.word_tokenize(prediction.lower())

        # Use smoothing function for BLEU
        smoothie = SmoothingFunction().method1
        bleu_score = sentence_bleu([reference_tokens], prediction_tokens, smoothing_function=smoothie)
        bleu_scores.append(bleu_score)

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0

    # ROUGE score
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for prediction, reference in zip(predictions, references):
        scores = rouge_scorer_instance.score(reference, prediction)
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    avg_rouge_scores = {
        'rouge1': sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1']),
        'rouge2': sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2']),
        'rougeL': sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL']),
    }

    # Final Results
    results = {
        'avg_bleu_score': avg_bleu_score,
        'avg_rouge1_score': avg_rouge_scores['rouge1'],
        'avg_rouge2_score': avg_rouge_scores['rouge2'],
        'avg_rougeL_score': avg_rouge_scores['rougeL'],
    }

    return results


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [52]:

# Example usage

predictions = [
     "On nutrition food labels, the energy from macronutrients is measured in calories, which is a unit of energy that represents the amount of heat required to raise a unit of mass of food from a temperature of 0°C to 37.3°C. Calories are the sum of the energy released when carbohydrates, proteins, and fats are broken down in the body.The difference between a calorie and a Calorie (with a capital C) is that Calories refer to the energy content of food, while the calorie (with a lowercase c) refers to the amount of heat required to raise a unit of mass of water from a temperature of 4°C to 100°C. This is why Calories are also known as thermochemical calories."

]
references = ["The energy is measured in Calories on food labels, which actually refers to kilocalories (1,000 calories). A calorie (small “c”) is a single unit of energy, while a Calorie (capital “C”) is equal to 1,000 small calories."

]

results = evaluate_rag_model(predictions, references)
print("Evaluation Results:")
print(f"Average BLEU Score: {results['avg_bleu_score']:.4f}")
print(f"Average ROUGE-1 Score: {results['avg_rouge1_score']:.4f}")
print(f"Average ROUGE-2 Score: {results['avg_rouge2_score']:.4f}")
print(f"Average ROUGE-L Score: {results['avg_rougeL_score']:.4f}")

Evaluation Results:
Average BLEU Score: 0.0587
Average ROUGE-1 Score: 0.3537
Average ROUGE-2 Score: 0.1605
Average ROUGE-L Score: 0.2439


In [53]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import numpy as np

# Function to calculate Precision, Recall, F1 Score for text comparison
def calculate_metrics(y_true, y_pred):
    """
    Calculate Precision, Recall, and F1 Score between ground truth (y_true) and model-generated (y_pred) text.

    Args:
    - y_true: list of reference text (ground truth)
    - y_pred: list of generated text (model output)

    Returns:
    - precision: Precision score
    - recall: Recall score
    - f1: F1 Score
    """
    # Tokenize and convert texts to a binary representation (1 if word exists in the text, else 0)
    y_true_tokens = [set(true.split()) for true in y_true]
    y_pred_tokens = [set(pred.split()) for pred in y_pred]

    # Flatten the sets of tokens into individual word lists for comparison
    all_words = set([word for sublist in y_true_tokens + y_pred_tokens for word in sublist])

    # Create binary vectors for precision, recall, and F1 score
    y_true_binary = [[1 if word in tokens else 0 for word in all_words] for tokens in y_true_tokens]
    y_pred_binary = [[1 if word in tokens else 0 for word in all_words] for tokens in y_pred_tokens]

    # Convert to numpy arrays for use in scikit-learn metrics
    y_true_array = np.array(y_true_binary)
    y_pred_array = np.array(y_pred_binary)

    # Calculate Precision, Recall, and F1 score using sklearn
    precision = precision_score(y_true_array, y_pred_array, average='micro')
    recall = recall_score(y_true_array, y_pred_array, average='micro')
    f1 = f1_score(y_true_array, y_pred_array, average='micro')

    return precision, recall, f1

In [54]:

# Example ground truth (reference) text
y_true = [
    "The energy is measured in Calories on food labels, which actually refers to kilocalories (1,000 calories). A calorie (small “c”) is a single unit of energy, while a Calorie (capital “C”) is equal to 1,000 small calories."

]

# Example generated text (model output)
y_pred = [
    "On nutrition food labels, the energy from macronutrients is measured in calories, which is a unit of energy that represents the amount of heat required to raise a unit of mass of food from a temperature of 0°C to 37.3°C. Calories are the sum of the energy released when carbohydrates, proteins, and fats are broken down in the body.The difference between a calorie and a Calorie (with a capital C) is that Calories refer to the energy content of food, while the calorie (with a lowercase c) refers to the amount of heat required to raise a unit of mass of water from a temperature of 4°C to 100°C. This is why Calories are also known as thermochemical calories."

]

precision, recall, f1 = calculate_metrics(y_true, y_pred)

# Calculate Precision, Recall, and F1 Score#
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.2698
Recall: 0.5152
F1 Score: 0.3542


In [55]:
!pip install bert-score
import bert_score

# Reference and generated sentences
reference = ["The energy is measured in Calories on food labels, which actually refers to kilocalories (1,000 calories). A calorie (small “c”) is a single unit of energy, while a Calorie (capital “C”) is equal to 1,000 small calories."]

generated = ["On nutrition food labels, the energy from macronutrients is measured in calories, which is a unit of energy that represents the amount of heat required to raise a unit of mass of food from a temperature of 0°C to 37.3°C. Calories are the sum of the energy released when carbohydrates, proteins, and fats are broken down in the body.The difference between a calorie and a Calorie (with a capital C) is that Calories refer to the energy content of food, while the calorie (with a lowercase c) refers to the amount of heat required to raise a unit of mass of water from a temperature of 4°C to 100°C. This is why Calories are also known as thermochemical calories."]

# Compute BERTScore
P, R, F1 = bert_score.score(generated, reference, lang="en")

# Print Precision, Recall, F1-score
print(f"Precision: {P.mean():.4f}")
print(f"Recall: {R.mean():.4f}")
print(f"F1-score: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8422
Recall: 0.8860
F1-score: 0.8635
